# Convert Categorical Variables

In [2]:
'''
HOW TO Encode categorical features as a one-hot numeric array. 
This creates a binary column for each (or k-1) category and returns a sparse matrix.
'''
import pandas as pd
df = pd.read_csv("../data/data_example.csv", sep = ",")
df

,type_of_food,some_other_attribute
0,fruit,0.3
1,vegetable,0.4
2,fruit,0.2
3,meat,0.3
4,fruit,0.6
5,vegetable,0.7
6,fruit,0.3
7,meat,0.4
8,vegetable,0.2
9,vegetable,0.3


In [3]:
# Convert categorical variable into dummy/indicator variables
dummy = pd.get_dummies(df['type_of_food'])   
#drop_first=True --> Whether to get k-1 dummies out of k categorical levels by removing the first level.

In [4]:
dummy

,fruit,meat,vegetable
0,True,False,False
1,False,False,True
2,True,False,False
3,False,True,False
4,True,False,False
5,False,False,True
6,True,False,False
7,False,True,False
8,False,False,True
9,False,False,True


- `pd.get_dummies(df['type_of_food'])`:

  - This function takes the type_of_food column and converts it into dummy (or one-hot encoded) variables.
  - Each unique category in the type_of_food column will be represented by a new binary column where:
       - 1 indicates the presence of that category.
       - 0 indicates the absence of that category.

- `drop_first=True`:

   - This argument is used to avoid the dummy variable trap by dropping the first category.
   - In one-hot encoding, if you have k categories, all k columns are often generated. This can cause multicollinearity in some models because the presence of one category can be inferred from the others. For example, if you know whether something is "fruit" and "vegetable," you automatically know if it is "meat."
    - By setting `drop_first=True`, you reduce the number of dummy variables from k to k-1, thus eliminating multicollinearity.

In [5]:
new_df = pd.concat([df,dummy], axis = 1) # axis = 1 concatanation on columns

In [6]:
new_df

,type_of_food,some_other_attribute,fruit,meat,vegetable
0,fruit,0.3,True,False,False
1,vegetable,0.4,False,False,True
2,fruit,0.2,True,False,False
3,meat,0.3,False,True,False
4,fruit,0.6,True,False,False
5,vegetable,0.7,False,False,True
6,fruit,0.3,True,False,False
7,meat,0.4,False,True,False
8,vegetable,0.2,False,False,True
9,vegetable,0.3,False,False,True


- `pd.concat()`: This function is used to concatenate two or more DataFrames along a particular axis (rows or columns). In this case, you're concatenating along the columns `(axis=1)`, meaning you're adding the dummy variables to the original DataFrame.

- `[df, dummy]`:

  - `df` is your original DataFrame, which contains the categorical `type_of_food` column and possibly other columns like `some_other_attribute`.
  - `dummy` is the DataFrame created by `pd.get_dummies()`, which contains the new binary columns representing the categories from `type_of_food`.

- axis=1:

- This argument tells `pd.concat()` to concatenate along the columns.
- So, the new DataFrame `new_df` will have all the original columns from `df` plus the one-hot encoded dummy columns from `dummy`.

In [7]:
new_df.drop("type_of_food", axis=1)

,some_other_attribute,fruit,meat,vegetable
0,0.3,True,False,False
1,0.4,False,False,True
2,0.2,True,False,False
3,0.3,False,True,False
4,0.6,True,False,False
5,0.7,False,False,True
6,0.3,True,False,False
7,0.4,False,True,False
8,0.2,False,False,True
9,0.3,False,False,True


## Sklearn OneHot encoder 

The `OneHotEncoder` from `sklearn.preprocessing` is another useful tool for converting categorical variables into a one-hot numeric array. Unlike `pd.get_dummies()`, it is more suitable for use in machine learning pipelines as it integrates directly with Scikit-learn estimators and can handle both training and test datasets consistently.

In [8]:
from sklearn.preprocessing import OneHotEncoder

In [9]:
encoder = OneHotEncoder(handle_unknown='ignore')


`handle_unknown='ignore'`:
- This parameter instructs the encoder to ignore any unknown categories that it encounters during transformation.

In [10]:
df[["type_of_food"]]

,type_of_food
0,fruit
1,vegetable
2,fruit
3,meat
4,fruit
5,vegetable
6,fruit
7,meat
8,vegetable
9,vegetable


In [11]:
encoder.fit(df[["type_of_food"]]) # Fit the encoder on the 'type_of_food' column


OneHotEncoder(handle_unknown='ignore')

In [12]:
encoder.categories_

[array(['fruit', 'meat', 'vegetable'], dtype=object)]

After fitting the `OneHotEncoder`, the `encoder.categories_` attribute contains the list of categories for each feature that was one-hot encoded. This is useful to see which unique categories the encoder has identified and encoded.

In [13]:
dummy = encoder.transform(df[["type_of_food"]]).toarray() # Fit the encoder on the 'type_of_food' column

In [14]:
dummy

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [15]:
df_categorical=pd.DataFrame(dummy,columns=encoder.categories_)

In [16]:
df_categorical

,fruit,meat,vegetable
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,0.0,1.0,0.0
4,1.0,0.0,0.0
5,0.0,0.0,1.0
6,1.0,0.0,0.0
7,0.0,1.0,0.0
8,0.0,0.0,1.0
9,0.0,0.0,1.0


In [17]:
encoded=[[0, 1, 0], [1, 0, 0]]

In [18]:
print(encoder.inverse_transform(encoded))


[['meat']
 ['fruit']]


This is a two-dimensional list (or a matrix) representing one-hot encoded data for a categorical feature with three possible categories. Each inner list corresponds to one observation (row), and each element in the inner list corresponds to a category.
Structure of the `encoded` List:
- First Observation: `[0, 1, 0]`
     - This indicates that the first observation belongs to the second category (e.g., `vegetable` if we consider the order to be `[fruit, vegetable, meat]`).
- Second Observation: `[1, 0, 0]`
     - This indicates that the second observation belongs to the first category (e.g., `fruit`).

In [19]:
import pickle

pickle.dump(encoder, open("ohe.pkl", 'wb'))

In [20]:
encoder = pickle.load(open("ohe.pkl", 'rb'))

In [21]:
print(encoder.inverse_transform(encoded))


[['meat']
 ['fruit']]


# Dictionaries

In [22]:
df

,type_of_food,some_other_attribute
0,fruit,0.3
1,vegetable,0.4
2,fruit,0.2
3,meat,0.3
4,fruit,0.6
5,vegetable,0.7
6,fruit,0.3
7,meat,0.4
8,vegetable,0.2
9,vegetable,0.3


In [31]:
mapping = {
    'fruit': 'plants',
    'vegetable': 'plants',
    'meat': 'animal'
}

df['new_type_of_food'] = df['type_of_food'].map(mapping)

In [35]:
df[['type_of_food','new_type_of_food']]

,type_of_food,new_type_of_food
0,fruit,plants
1,vegetable,plants
2,fruit,plants
3,meat,animal
4,fruit,plants
5,vegetable,plants
6,fruit,plants
7,meat,animal
8,vegetable,plants
9,vegetable,plants


In [36]:
mapping = {
    'fruit': 1,
    'vegetable': 2,
    'meat': 3
}

df['num_type_of_food'] = df['type_of_food'].map(mapping) # 

In [37]:
df[['type_of_food','num_type_of_food']]

,type_of_food,num_type_of_food
0,fruit,1
1,vegetable,2
2,fruit,1
3,meat,3
4,fruit,1
5,vegetable,2
6,fruit,1
7,meat,3
8,vegetable,2
9,vegetable,2


For more advanced ordinal encodings: [https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html) (but make sure you are in control)